In [ ]:
# Introduction to Python Midterm Project
# First-grade Math Electronic Flash Cards: Math1Flash
# Author: Steve DeGrange
#
# Math1Flash is an electronic flash card educational game program.
#
# With physical cards, you have a stack of cards with a problem
# on the front side and the answer on the reverse side. A card is
# drawn and you are shown the front side for you to answer. if
# you don't get the answer right, you are shown the reverse side.
#
# In Math1Flash, you (intended to be a first-grader learning basic math)
# math) are given a random arithetic or number compare problem to answer
# (the front side) and if you need you're shown the correct answer
# (the reverse side).  This is then repeated (the card stack) with
# a new random problem, until you choose to quit.
#
# Examples: What is 9x11?             (arithmetic)
#           Is 10 > 16?  yes or no    (number compare)
#
# If you answer correctly, you are rewarded with praise and
# asked to try another problem.
#
# If you don't answer correctly, you are given up to two more
# tries to get it right and then told the answer.
# Before each retry you're given a "hint" such as "That's too low"
# or "That's not yes or no".
# You can skip problems/retries by just entering "" instead of an
# answer if you want another question instead, or to quit.
#
# This is repeated until you don't reply affirmatively to getting
# another problem/question.
#
# In addition to entering "" to skip, you can abbreviate
# "yes" and "no" down to one letter at all prompts.

import random

RED, GREEN, BLUE, BOLD, END = "\033[0;31m\033[1m","\033[0;32m\033[1m","\033[0;34m\033[1m", "\033[1m", "\033[0m"    # ANSI codes
YESNO = ["yes", "ye", "y", "no", "n"]  # Valid replies to yes/no prompts

# Math operator choices
ARITHMETIC = ["+", "-", "x"]                  # Supported arithmetic operators
COMPARISON = ["=", "≠","<", ">", "≤", "≥" ]   # Supported comparison operators

# Function getOperator returns the randomly chosen allowed operator
# It also returns the type of that operator: "arithmetic" or "compare numbers"
def getOperator():
    return random.choice(ARITHMETIC + ARITHMETIC + COMPARISON)   # double arith because 2x comp ops

# Function getOperands returns a pair of random non-negative integers for use as operands
def getOperands(operator):
    max = 10 if operator == "x" else 40  # cut back for multiplication
    return [random.randint(0, max), random.randint(0, max)]

# Function getQuestion returns the current problem as a question, such as "What is 6 x 9?"
def getQuestion(operator, operand1, operand2):
    # question is a 3-part array: [1] is the main part, [0] is a prolog, [2] is an epilog
    question = ["", "", ""]
    question[1] = "%d %s %d" % (operand1, operator, operand2)
    if operator in ARITHMETIC:
        question[0] = "What is "
        question[2] = "?   (answer a number)"
    else:
        question[0] = "Is "
        question[2] = "?   (answer yes or no)"
    return question
    
# Function getAnswer returns the answer to the current problem, such as "54"
def getAnswer(operator, operand1, operand2):
    # answer is a 2-part array: [0] is the internal form of the answer, [1] is the string form of the answer
    answer = [None, ""]
    match operator:
        case "+":
            answer[0] = operand1 + operand2
        case "-":
            answer[0] = operand1 - operand2
        case "x":
            answer[0] = operand1 * operand2
        case "=":
            answer[0] = operand1 == operand2
        case "≠":
            answer[0] = operand1 != operand2
        case "<":
            answer[0] = operand1 < operand2
        case ">":
            answer[0] = operand1 > operand2
        case "≤":
            answer[0] = operand1 <= operand2
        case "≥":
            answer[0] = operand1 >= operand2
        case _:
            return answer
    if operator in ARITHMETIC:
        answer[1] = str(answer[0])
    else:
        answer[1] = "yes" if answer[0] else "no"
    return answer
   
# Function getProblem returns question and answer for of a problem given the operator and operands
def getProblem(operator, operand1, operand2):
    question = getQuestion(operator, operand1, operand2)
    answer = getAnswer(operator, operand1, operand2)
    return [question, answer]

# Function getHint returns text for hinting how to correct an incorrect answer
def getHint(answerUser, answer, operator):
    if operator in COMPARISON:
        return "not yes or no" if not answerUser in YESNO else "wrong, but you'll get it"
    if not answerUser.isdigit():
        return "not a whole number"
    return "not big enough" if int(answerUser) < answer[0] else "too big"

# Function validateReply returns whether an answer from the user is correct or incorrect
def validateAnswer(answerUser, answer, operator):
    if operator in ARITHMETIC:
        return True if answerUser.isdigit() and int(answerUser) == answer[0] else False
    return True if answerUser != "" and answer[1].startswith(answerUser) else False

# Function getYesNo asks for "yes" or "no" (or "" for default) or a shortening like "n" or "ye".
def getYesNo(prompt, default = "yes"):
    reply = input(prompt).lower()
    if reply == "":
        reply = default
    while not reply in YESNO:
        reply = input("Please say yes or no. Try again.\n ").lower()
        if reply == "":
            reply = default
    return True if reply[0] == "y" else False


#------------------#
#     mainline     #
#------------------#
# Initialization
attempts, correct = 0, 0
game = "Flash"
user = input(f"Hi! My name is {game}.  What's yours? ")
if user == "":
    user = "Mathematician"
print(f"I'm pleased to meet you, {BOLD}{user}{END}! I think we may have met before?\n")
print("It's time for some math questions!")
print(f"Some are {BLUE}arithmetic{END} and some {GREEN}compare numbers{END}.")
continueGame = getYesNo("\nAre you ready to start? ")

# Present new problems as long as they want
while continueGame:
    # Build the next question/answer pair
    operator = getOperator()
    opColor = BLUE if operator in ARITHMETIC else GREEN
    operand1, operand2 = getOperands(operator)
    question, answer = getProblem(operator, operand1, operand2)

    # If multiplication and the answer is too big, or subtraction and the answer is negative
    #   then loop getting new operands, but keeping operator, until the problem is easy enough
    while (operator == 'x' and answer[0] > 100) or (operator == '-' and answer[0] < 0):
        operand1, operand2 = getOperands(operator)
        question, answer = getProblem(operator, operand1, operand2)
    
    # Present the question to the user and ask for the answer, repeating up to twice more.
    # A reply of "" anywhere indicates to skip this question and move on to another (or the end).
    opType = "aritmetic" if operator in ARITHMETIC else "compare numbers"
    print(f"\nTry this {opColor}{opType}{END} problem:")
    print(f"   {question[0]}{opColor}{question[1]}{END}{question[2]}")
    reply = input("   ")
    if reply != "":
        attempts += 1
        # See if they got it right the first time
        if validateAnswer(reply, answer, operator):
            correct += 1
            # Pick a random kudos phrase so its not too boring
            match random.randint(0, 4):
                case 0:
                    print(f"\n{RED}Right! You're really super at this {user}!{END}")
                case 1:
                    print(f"\n{RED}Correct! You're doing great, {user}!{END}")
                case 2:
                    print(f"\n{RED}You got it, {user}! You're doing really good!{END}")
                case 3:
                    print(f"\n{RED}Nice job, {user}! You're super at this!{END}")
                case _:
                    print(f"\n{RED}That's right! Keep it up, {user}!{END}")

        # Allow second and third chances (or skipping this problem if "" entered as reply)
        elif reply != "":           # Use "" to mean skip this problem and move on        
            # Second chance or skip this problem if reply is ""
            reply = input(f"\nThat's {getHint(reply, answer, operator)}. Try again!\n   ")
            if validateAnswer(reply, answer, operator):
                correct += 1        # We're easy graders... count it as ok since they got it 2nd time
                print(f"\n{RED}Correct! You got it, {user}!{END}")

            # Third and last chance or skip this problem if reply is ""
            elif reply != "":       # Use "" to mean skip this problem and move on
                reply = input(f"\nThat's {getHint(reply, answer, operator)}. Try once more.\n   ")
                if validateAnswer(reply, answer, operator):
                    correct += 1    # We're easy graders... count it as ok since they got it 3rd time
                    print(f"\n{RED}Yes, you did it!!  Good job, {user}!{END}")
                elif reply != "":   # Use "" to mean skip this problem and move on
                    # Incorrect 3 times.  You can't win them all
                    print(f"\nThat's still not right. It was hard.\nThe answer was {opColor}{answer[1]}{END}.")

    if reply == "":  # Problem skipped by user entering "" as their answer
        print(f"OK, we'll skip that. \nThe answer was {opColor}{answer[1]}{END}.") 
    
    # See if they want to get a new problem or just quit
    continueGame = getYesNo("\nDo you want a new problem?\n Say yes or no.\n")

# All done. Print some ending messages
if attempts > 1 and correct > attempts / 2:
    if correct == attempts:
        print(f"{RED}You didn't miss any! {correct} out of {attempts}! Most excellent!{END}")
    else:
        print(f"{RED}You got {correct} out of {attempts} right! Nice job!{END}")

print(f"Bye, {BOLD}{user}{END}. Come back soon.")
